## Final Submitted Code Review

In [1]:
from collections import defaultdict
import json
import sys
import time

In [26]:
# path = sys.argv[1]

path = input("Enter path to JSON file or file name:")
with open(path, "r") as dag_file: 
    try: 
        input_dag = json.loads(dag_file.read()) #make sure the input it properly formated in JSON
        # adj_list, root = make_adjacency_list() #make an adjacency list with the input file
        
        # tasks = TaskRunner(adj_list, root)
        # checked_sorted_tasks = check_acylic_sort(adj_list, root) #check if graph is acyclic and make sorted list of tasks

        # if checked_sorted_tasks == False:
        #     print("Task runner can not execute a list of tasks that has circular dependencies.")
        # else: 
        #     run_tasks(checked_sorted_tasks) #print out the tasks in the appropriate order and time differential

    except ValueError as e: 
        print(f"JSON file is invalid. Please check input. Error: {e}")

In [30]:
print(input_dag)

{'A': {'start': True, 'edges': {'B': 3, 'C': 2, 'D': 1}}, 'B': {'edges': {'E': 5}}, 'C': {'edges': {'E': 6}}, 'D': {'edges': {'E': 4}}, 'E': {'edges': {'F': 8, 'G': 9}}, 'F': {'edges': {}}, 'G': {'edges': {}}}


### Make Adjacency List/Dictionary 

Formal Definition: a collection of unordered lists used to represent a finite graph. 

Pros: 

-Helps find all vertices adjacent to a vertex easily. 

The input dag from the JSON is basically structured as an adjacency list. I want to clean it up and make it easier to use for downstream functions. 

In [31]:

'''
Make a weighted adjacency list with the dict converted from the json input.
'''
adj_list = defaultdict(list)
start_node = ''

for key in input_dag:
    if len(input_dag[key]) == 2: #with the assumption that there is only one start node specified, the length of that key will be 2
        start_node = key 
    
    if input_dag[key]['edges'] != {}: #for verices that have children/dependent tasks
        for nested_key, nested_value in input_dag[key]['edges'].items():
            adj_list[key].append((nested_key, nested_value)) #add weights and edges of the children in the form of a tuple 
    else: 
        adj_list[key].append(None) #if there are no children, assign the edges NoneType

print(f"Adjacency list: {adj_list}", f"\nStart Vertex: {start_node}")
# return adj_list, start_node

A
B
C
D
E
F
G
Adjacency list: defaultdict(<class 'list'>, {'A': [('B', 3), ('C', 2), ('D', 1)], 'B': [('E', 5)], 'C': [('E', 6)], 'D': [('E', 4)], 'E': [('F', 8), ('G', 9)], 'F': [None], 'G': [None]}) 
Start Vertex: A


In [34]:
''' 
Determine whether there are cycles in the DAG using a BFS traversal approach and populate/sort a dictioanary 
with the nodes and the order and time in which they will be printed. I would consider this pre-processing.
'''
num_incoming_vert = defaultdict(int) #create a dictionary to track the number of dependencies of each task 
visited = defaultdict(bool) #create dictionary to ensure all vertices are visited

for vertex in adj_list.keys():
    num_incoming_vert[vertex] #populate the dict
    visited[vertex]

for vertex in adj_list.keys():
    print(f"Vertex: {vertex}")
    for inc_edge in adj_list[vertex]: #iterate through the dependents of a task. adj_list[vertex] is the list of tuples for the vertex
        print(f"In {adj_list[vertex]}")
        if inc_edge != None:
            num_incoming_vert[inc_edge[0]] += 1 #if a dependent shows up in the adjacency, increment it's number of dependencies by 1
        else: 
            pass

Vertex: A
In [('B', 3), ('C', 2), ('D', 1)]
In [('B', 3), ('C', 2), ('D', 1)]
In [('B', 3), ('C', 2), ('D', 1)]
Vertex: B
In [('E', 5)]
Vertex: C
In [('E', 6)]
Vertex: D
In [('E', 4)]
Vertex: E
In [('F', 8), ('G', 9)]
In [('F', 8), ('G', 9)]
Vertex: F
In [None]
Vertex: G
In [None]


In [19]:
queue = []
queue.append(start_node) #build queue and append the start node. 
time_for_node = defaultdict(int) #record time for each node
time_for_node[start_node]

#populate the queue with any other tasks that dont have any dependences 
for node in num_incoming_vert: 
    if num_incoming_vert[node] == 0 and not start_node:
        queue.append(node)

#start working through the queue to determine if there are any cycles 

while(queue):
    current_node = queue.pop(0) #FIFO approach 

    #if all vertices in queue have been visited and if the vertex doesn't have any dependents, pop it from the queue
    if adj_list[current_node] == [None] and num_incoming_vert[current_node] == 0:
        visited[current_node] = True
    elif visited[current_node] == False: #if the vert has dependents, cycle through all of them and update the number of incoming verts by 1 bc they have now been visited
        for vert,weight in adj_list[current_node]:
            for node in vert: 
                num_incoming_vert[node[0]] -= 1
                time_for_node[node[0]] = time_for_node[current_node] + weight #populate the dict with the time a node will be printed
            if num_incoming_vert[node[0]] == 0: 
                queue.append(node[0])

            visited[current_node] = True

check = len(set(visited.values())) == True

if check == False: #the number of visited vertices has to be equal to the number of vertices in the task runner for it to be acyclic. 
    print(False)
else: 
    sorted_tasks = sorted(time_for_node.items(), key = lambda x:x[1]) #sort the time dict in ASC
    print(sorted_tasks)

In [20]:
start_time = time.time() #Time starts now!! 

for node in sorted_tasks: 
    if round(time.time() - start_time, 1) >= node[1]: #check if the time elapsed is equal to the time of the given node
        print(node[0])
    elif round(time.time() - start_time, 1) < node[1]: #if the right amount of time has not been elapsed, wait 
        time.sleep(node[1] - round(time.time() - start_time, 1))
        print(node[0])